## Scraping countries data and boundaries

Link to data from Natural Earth: [counties data](https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/ne_50m_admin_0_countries.zip)



In [29]:
import geopandas as gpd

In [10]:
countries_shp = gpd.read_file("../temp/country/ne_50m_admin_0_countries.shp", layer='ne_50m_admin_0_countries')

In [12]:
countries_shp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 242 entries, 0 to 241
Columns: 162 entries, featurecla to geometry
dtypes: float64(4), geometry(1), int64(25), object(132)
memory usage: 306.4+ KB


In [31]:
countries_shp.columns

Index(['featurecla', 'scalerank', 'LABELRANK', 'SOVEREIGNT', 'SOV_A3',
       'ADM0_DIF', 'LEVEL', 'TYPE', 'ADMIN', 'ADM0_A3',
       ...
       'FCLASS_TR', 'FCLASS_ID', 'FCLASS_PL', 'FCLASS_GR', 'FCLASS_IT',
       'FCLASS_NL', 'FCLASS_SE', 'FCLASS_BD', 'FCLASS_UA', 'geometry'],
      dtype='object', length=162)

In [27]:
filter_countries = countries_shp[["NAME", "NAME_LONG", "FIPS_10", "ISO_A2", "ISO_A3", "POSTAL", "TYPE", "CONTINENT", "SUBREGION", "WIKIDATAID" ]]

In [28]:
filter_countries

,NAME,NAME_LONG,FIPS_10,ISO_A2,ISO_A3,POSTAL,TYPE,CONTINENT,SUBREGION,WIKIDATAID
0,Zimbabwe,Zimbabwe,ZI,ZW,ZWE,ZW,Sovereign country,Africa,Eastern Africa,Q954
1,Zambia,Zambia,ZA,ZM,ZMB,ZM,Sovereign country,Africa,Eastern Africa,Q953
2,Yemen,Yemen,YM,YE,YEM,YE,Sovereign country,Asia,Western Asia,Q805
3,Vietnam,Vietnam,VM,VN,VNM,VN,Sovereign country,Asia,South-Eastern Asia,Q881
4,Venezuela,Venezuela,VE,VE,VEN,VE,Sovereign country,South America,South America,Q717
...,...,...,...,...,...,...,...,...,...,...
237,Afghanistan,Afghanistan,AF,AF,AFG,AF,Sovereign country,Asia,Southern Asia,Q889
238,Siachen Glacier,Siachen Glacier,-99,-99,-99,SG,Indeterminate,Asia,Southern Asia,Q333946
239,Antarctica,Antarctica,AY,AQ,ATA,AQ,Indeterminate,Antarctica,Antarctica,Q51
240,Sint Maarten,Sint Maarten,NT,SX,SXM,SX,Country,North America,Caribbean,Q26273
